## Overview
The preprocessing steps were implemented in a modular Jupyter Notebook and follow a structured ETL (Extract, Transform, Load) approach:

### Extract

- Data is sourced from CSV files downloaded from Statistics Canada and other official open data sources.
- The datasets include:
  - Tuition costs (2006–2025)
  - Graduation counts by field and education level
  - Median income five years post-graduation
  - Labour market outcomes from the census (2011, 2016, 2021)

### Transform

- **Tuition Data**: Renamed columns, extracted the starting year from academic ranges (e.g., "2010/2011" becomes 2010), and filtered out aggregate categories. Education level was added to each row.
- **Graduation Data**: Cleaned field names, unified education levels (Bachelor's, Master's, Doctoral → Undergraduate, Graduate), and filtered for Canadian-wide data.
- **Income Data**: Removed summary categories (e.g., STEM, BHASE), standardized field names, and derived `Graduation Year` from `Survey Year - 5`.
- **Labour Data**: Parsed raw files for census years, removed metadata rows, standardized columns, and cleaned up annotations from field names.
- **Field of Study Normalization**: Applied a universal mapping across datasets to ensure consistent field names.
- **Education Level Unification**: For field-level analysis, education types were merged under a single "Postsecondary" category.
- **Aggregation**:
  - Graduates were summed per year and field.
  - Tuition was computed as a weighted average based on graduate counts.
  - Median income values were retained at the field-year level (not education-specific).
- **Census Alignment**: Each observation year was mapped to its nearest census year (2001, 2006, 2011, 2016, 2021, or 2026) to match labour force data.

### Load

- Cleaned and aggregated datasets were exported as CSV files for downstream analysis:
  - `postsecondary_field_outcomes.csv`: Combined tuition, graduates, and income data by field and year.
  - `postsecondary_labour_outcomes.csv`: Census-based labour indicators by field of study.

In [1]:
import pandas as pd
import re
import os

## Extract data

### Tution Data
This dataset offers annual data on undergraduate tuition fees across various fields of study in Canada, measured in current dollars. The data spans academic years from 2006/2007 to 2024/2025. The data is sourced from the Tuition and Living Accommodation Costs survey. Notably, a correction was made on October 21, 2020, to address an error affecting the 2019/2020 and 2020/2021 data for one Alberta college, impacting average tuition fees for Alberta and Canada in certain fields. We focus on the broader national average.

In [2]:
def preprocess_tuition_data(df, edu_level):
    # Select and rename relevant columns
    cols = ["REF_DATE", "Field of study", "VALUE"]
    df_ = df[cols].copy()
    df_ = df_.rename(columns={"REF_DATE": "Year", "VALUE": "Tuition"})
    
    # Extract start year
    df_["Year"] = df_["Year"].apply(lambda x: x.split('/')[0])
    df_["Year"] = df_["Year"].astype(int)
    
    # Remove total fields
    df_ = df_[df["Field of study"] != "Total, field of study"]
    
    # Set education level
    df_["Education"] = edu_level
    
    return df_

In [3]:
undergrad_tuition_data = pd.read_csv("data/tuition/undergraduate.csv")
grad_tuition_data = pd.read_csv("data/tuition/graduate.csv")

In [4]:
tuition_data = pd.concat([
    preprocess_tuition_data(undergrad_tuition_data, edu_level="Undergraduate"), 
    preprocess_tuition_data(grad_tuition_data, edu_level="Graduate")
])

In [5]:
tuition_data.head()

,Year,Field of study,Tuition,Education
19,2006,Education,3373.0,Undergraduate
20,2007,Education,3545.0,Undergraduate
21,2008,Education,3652.0,Undergraduate
22,2009,Education,3739.0,Undergraduate
23,2010,Education,3850.0,Undergraduate


### Median Income Data
This dataset provides information on the characteristics and median employment income of postsecondary graduates in Canada five years after graduation. It includes data from 2010 to 2017, covering graduates by educational qualification and field of study, grouped into STEM and BHASE (non-STEM) categories. The data is sourced from the Postsecondary Student Information System and the T1 Family File. Our focus is on the broader national trends rather than provincial or demographic breakdowns.

In [6]:
def preprocess_income_data(df):
    education_cred_mapping = {
        "Undergraduate certificate": "Bachelor’s or equivalent",
        "Undergraduate diploma": "Bachelor’s or equivalent",
        "Undergraduate degree": "Bachelor’s or equivalent",
        "Undergraduate associate": "Bachelor’s or equivalent",
        "Other undergraduate credential": "Bachelor’s or equivalent",
        "Post-baccalaureate non-graduate degree": "Bachelor’s or equivalent",
        "Other post-baccalaureate non-graduate credential": "Bachelor’s or equivalent",
        "Undergraduate-level short credential": "Bachelor’s or equivalent",
        "Health-related residency program diploma": "Master's or equivalent",
        "Health-related residency program certificate": "Master's or equivalent",
        "Health-related residency program degree": "Master's or equivalent",
        "Master's certificate": "Master's or equivalent",
        "Master's diploma": "Master's or equivalent",
        "Master's degree": "Master's or equivalent",
        "Other master's-level credential": "Master's or equivalent",
        "Master's-level short credential": "Bachelor’s or equivalent",
        "Doctoral-level certificate": "Doctoral or equivalent",
        "Doctoral-level diploma": "Doctoral or equivalent",
        "Doctoral degree": "Doctoral or equivalent",
        "Post=doctoral diploma": "Doctoral or equivalent"
    }

    fields = [
        "Education [1]", "Visual and performing arts, and communications technologies [2]",
        "Humanities [3]", "Social and behavioural sciences and law [4]", 
        "Business, management and public administration [5]", 
        "Physical and life sciences and technologies [6]",
        "Mathematics, computer and information sciences [7]", 
        "Architecture, engineering, and related technologies [8]",
        "Agriculture, natural resources and conservation [9]",
        "Health and related fields [10]",
        "Personal, protective and transportation services [11]"
    ]
    
    # Filter rows
    df_ = df[
        (df["GEO"] == "Canada")
        & (df["Field of study"].isin(fields))
        & (df["Educational qualification"].isin(education_cred_mapping.keys()))
        & (df["Gender"] == "Total, gender")
        & (df["Age group"] == "15 to 64 years")
        & (df["Status of student in Canada"] == "Canadian and international students")
        & (df["Characteristics after graduation"] == "Graduates reporting employment income")
        & (df["Graduate statistics"] == "Median employment income")
    ].copy()

    assert len(df_["GEO"].unique()) == 1 and \
           len(df_["Gender"].unique()) == 1 and \
           len(df_["Age group"].unique()) == 1 and \
           len(df_["Status of student in Canada"].unique()) == 1 and \
           len(df_["Characteristics after graduation"].unique()) == 1 and \
           len(df_["Graduate statistics"].unique()) == 1

    # Select and rename relevant columns
    cols = ["REF_DATE", "Educational qualification", "Field of study", "VALUE"]
    df_ = df_.rename(columns={
        "REF_DATE": "Graduation Year", 
        "VALUE": "Median income", 
        "Educational qualification": "Education"
    })

    # Clean up field of study names by removing trailing brackets
    df_["Field of study"] = df_["Field of study"].apply(lambda x: re.sub(r'\s+\[\d+\]', "", x))

    # Normalize education levels
    df_["Education"] = df_["Education"].replace(education_cred_mapping)
    
    # Aggregate by field of study, education level, and survey year
    df_ = df_.groupby(["Graduation Year", "Field of study", "Education"], as_index=False)["Median income"].median()

    # Add survey year (2 years after graduation year)
    df_["Survey Year"] = df_["Graduation Year"] + 2
    
    return df_

In [7]:
raw_income_data = pd.read_csv("data/income/median_income_after_2_years.csv")
raw_income_data.shape

(5454852, 21)

In [8]:
income_data = preprocess_income_data(raw_income_data)

In [9]:
income_data.head()

,Graduation Year,Field of study,Education,Median income,Survey Year
0,2010,"Agriculture, natural resources and conservation",Bachelor’s or equivalent,47100.0,2012
1,2010,"Agriculture, natural resources and conservation",Doctoral or equivalent,62600.0,2012
2,2010,"Agriculture, natural resources and conservation",Master's or equivalent,68000.0,2012
3,2010,"Architecture, engineering, and related technol...",Bachelor’s or equivalent,69800.0,2012
4,2010,"Architecture, engineering, and related technol...",Doctoral or equivalent,89600.0,2012


### Labour Data
This combined dataset draws from Canadian census and National Household Survey data (2011, 2016, and 2021) to provide a cross-sectional view of the labour force status by major field of study, highest certificate, diploma or degree, and demographic characteristics such as age and gender. Each dataset reflects the population aged 15 years and over in private households at the national level. The data are sourced from the 25% sample of the census and NHS populations. By aligning these datasets, we analyze trends in educational attainment and employment outcomes across time. Our focus remains on national-level comparisons across years and fields of study.

In [10]:
def preprocess_labour_data(filepath, year, version=1):
    if version == 1:
        # Skip metadata rows
        with open(filepath, "r", encoding="utf-8") as f:
            lines = f.readlines()
            
        # locate header row
        for i,line in enumerate(lines):
            if line.strip().startswith('"Major field of study') or line.strip().startswith('Major field of study'):
                table_start = i
                break
                    
        # Locate footer rows
        for j in range(len(lines)-1, -1, -1):
            if lines[j].strip().startswith('"Note') or lines[j].strip().startswith('Abbreviation notes:'):
                table_end = j
                break
        
        # Extract the actual table lines
        table_lines = lines[table_start:table_end]
        
        # Load the CSV from those lines
        from io import StringIO
        df = pd.read_csv(StringIO('\n'.join(table_lines)), encoding="utf-8")
        
        # Rename first column for easier reference
        df.columns.values[0] = "Field of study"
        
        # Define regex pattern to match main fields
        pattern = re.compile(r"^\s*\d")
        df = df[~df["Field of study"].astype(str).str.strip().str.match(pattern)]

        df.columns = [col.strip() for col in df.columns]
        
        # Extract relevant columns
        cols = [
            "Field of study", "Participation rate", "Employment rate", "Unemployment rate",
            "In the labour force", "Employed", "Unemployed", "Not in the labour force"
        ]
        df = df[cols]
        
        # Add Year column
        df["Year"] = year

        # Drop totals and calculated rows
        df = df.iloc[2:]

        # Clean up text
        df = df[~df["Field of study"].isnull()]
        df["Field of study"] = df["Field of study"].apply(lambda x: re.sub(r'\s+\[\d+\]', "", x))
        df["Field of study"] = df["Field of study"].str.strip()
        
        # Reset index
        df.reset_index(drop=True, inplace=True)
    elif version == 2:
        # Select relevant columns
        cols = ["REF_DATE", "Field of study", "Labour force status (8)"]
        df = pd.read_csv(filepath)
        
    else:
        raise Exception("Version not supported.")
  
    return df

In [11]:
labour_data = pd.concat([
    preprocess_labour_data(filepath="data/labour/2021.csv", year=2021, version=1),
    preprocess_labour_data(filepath="data/labour/2016.csv", year=2016, version=1),
    preprocess_labour_data(filepath="data/labour/2011.csv", year=2011, version=1)
])

labour_data.reset_index(drop=True, inplace=True)

In [12]:
labour_data.head()

,Field of study,Participation rate,Employment rate,Unemployment rate,In the labour force,Employed,Unemployed,Not in the labour force,Year
0,Education,62.2,59.2,4.9,"692,215","658,610","33,605","420,805",2021
1,"Visual and performing arts, and communications...",75.9,67.2,11.5,"479,040","424,020","55,015","152,220",2021
2,Humanities,68.3,61.7,9.6,"632,770","572,110","60,660","293,790",2021
3,Social and behavioural sciences and law,76.5,70.3,8.2,"1,542,740","1,416,630","126,110","473,385",2021
4,"Business, management and public administration",75.1,69.3,7.7,"2,822,835","2,604,645","218,195","935,415",2021


In [13]:
labour_data["Field of study"] = labour_data["Field of study"].replace({
    "Other 18": "Other",
    "Other fields of study": "Other",
    "Architecture, engineering, and related trades": "Architecture, engineering, and related technologies"
})

### Graduation Data
This dataset offers annual counts of postsecondary graduates in Canada, categorized by institution type (e.g., university, college), education level (e.g., bachelor's, master's, doctoral degrees), detailed field of study, gender, and student status in Canada (domestic or international). The data spans from 2000 to 2022 and is sourced from the Postsecondary Student Information System (PSIS). It provides insights into trends in postsecondary education completions across various disciplines and demographic groups. Our analysis focuses on national-level trends across different fields of study and education levels.

In [14]:
def preprocess_graduation_data(filepath):
    df = pd.read_csv(filepath)

    # Select and rename relveant columns
    cols = ["REF_DATE", "Field of study", "VALUE", "International Standard Classification of Education (ISCED)", "GEO"]
    df = df[cols]
    df = df.rename(columns={
        "REF_DATE": "Year", 
        "VALUE": "Graduates",
        "International Standard Classification of Education (ISCED)": "Education"
    })

    df = df[df["GEO"] == "Canada"]

    # Clean up field of study text
    df["Field of study"] = df["Field of study"].apply(lambda x: re.sub(r'\s+\[\d+\]', "", x))

    df = df.drop(columns=["GEO"])

    return df

In [15]:
def find_csv_files(directory):
    csv_files = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(".csv"):
            csv_files.append(os.path.join(directory, filename))
    return csv_files

In [16]:
csv_files = find_csv_files("data/graduation")

In [17]:
graduation_data = pd.concat([preprocess_graduation_data(filepath) for filepath in csv_files])

In [18]:
graduation_data.head()

,Year,Field of study,Graduates,Education
0,2000,"Business, management and public administration",19836.0,Bachelor's or equivalent
1,2001,"Business, management and public administration",21198.0,Bachelor's or equivalent
2,2002,"Business, management and public administration",22857.0,Bachelor's or equivalent
3,2003,"Business, management and public administration",24324.0,Bachelor's or equivalent
4,2004,"Business, management and public administration",26118.0,Bachelor's or equivalent


## Transform data

### Reconcile `Field of study` Categories
Since our analysis centers on the `Field of study`, it's crucial to standardize terminology across all datasets. Each dataset uses slightly different labels, so we must align them under a unified set of categories to enable accurate comparisons and analysis.

To ensure consistency across all datasets, we select the Graduation data's Field of study categories as our standard reference. This dataset offers a concise and balanced list of academic fields that are broad enough to encompass the terminology used in the other datasets. We may then use a dictionary-based mapping approach to align the field names from Tuition, Income, and Labour data to this standardized list. This method would be more appropriate than NLP techniques due to the small number of distinct categories and the categorical nature of the data, making direct mapping more reliable and transparent.

In [19]:
graduation_data["Field of study"] = graduation_data["Field of study"].replace({"Humanities": "Arts and humanities"})

In [20]:
graduation_data["Field of study"].value_counts()

Field of study
Business, management and public administration        69
Education                                             69
Architecture, engineering and related technologies    69
Health and related fields                             69
Arts and humanities                                   69
Mathematics, computer and information sciences        69
Social and behavioural sciences and law               69
Name: count, dtype: int64

In [21]:
# Standardized field list from graduation data
standard_fields = [
    "Business, management and public administration",
    "Education",
    "Architecture, engineering and related technologies",
    "Health and related fields",
    "Humanities",
    "Mathematics, computer and information sciences",
    "Social and behavioural sciences and law"
]

In [22]:
# Mapping dictionaries
field_mapping = {
    "Architecture": "Architecture, engineering and related technologies",
    "Engineering": "Architecture, engineering and related technologies",
    "Architecture, engineering, and related technologies": "Architecture, engineering and related technologies",
    "Mathematics, computer and information sciences": "Mathematics, computer and information sciences",
    "Mathematics and computer and information sciences": "Mathematics, computer and information sciences",
    "Health care": "Health and related fields",
    "Medicine": "Health and related fields",
    "Nursing": "Health and related fields",
    "Dentistry": "Health and related fields",
    "Veterinary medicine": "Health and related fields",
    "Pharmacy": "Health and related fields",
    "Other health, parks, recreation and fitness": "Health and related fields",
    "Humanities": "Arts and humanities",
    "Arts and humanities": "Arts and humanities",
    "Visual and performing arts, and communications technologies": "Arts and humanities",
    "Social and behavioural sciences, and legal studies": "Social and behavioural sciences and law",
    "Social and behavioural sciences and law": "Social and behavioural sciences and law",
    "Social and behavioural sciences": "Social and behavioural sciences and law",
    "Social and behavioural sciences, and legal studies": "Social and behavioural sciences and law",
    "Law": "Social and behavioural sciences and law",
    "Legal professions and studies": "Social and behavioural sciences and law",
    "Business and administration": "Business, management and public administration",
    "Business, management and public administration": "Business, management and public administration",
    "Education and teaching": "Education",
    "Education": "Education",
    "Physical and life sciences and technologies": "Architecture, engineering and related technologies",
    "Science and science technology": "Architecture, engineering and related technologies",
    "Agriculture, natural resources and conservation": "Architecture, engineering and related technologies",
    "Trades, services, natural resources and conservation": "Architecture, engineering and related technologies",
    "Personal, protective and transportation services": "Architecture, engineering and related technologies",
    "Executive MBA": "Business, management and public administration",
    "Regular MBA": "Business, management and public administration",
    "Optometry": "Health and related fields",
    "Other": "Other"
}

In [23]:
def normalize_field(df, values):
    df["Field of study"] = df["Field of study"].map(field_mapping).fillna(df["Field of study"])
    df = df[df["Field of study"].isin(values)].copy()

    # Simplify field of study titles
    df["Field of study"] = df["Field of study"].replace({
        "Architecture, engineering and related technologies": "Architecture & Engineering",
        "Arts and humanities": "Arts and Humanities",
        "Business, management and public administration": "Business & Public Admin",
        "Education": "Education",
        "Health and related fields": "Health",
        "Mathematics, computer and information sciences": "Mathematics & CIS",
        "Social and behavioural sciences and law": "Social Sciences and Law"
    })
    
    return df

In [24]:
graduation_fields = graduation_data["Field of study"].unique()

In [25]:
tuition_data = normalize_field(tuition_data, graduation_fields)
income_data = normalize_field(income_data, graduation_fields)
labour_data = normalize_field(labour_data, graduation_fields)
graduation_data = normalize_field(graduation_data, graduation_fields)

In [26]:
graduation_data["Field of study"].value_counts().sort_index()

Field of study
Architecture & Engineering    69
Arts and Humanities           69
Business & Public Admin       69
Education                     69
Health                        69
Mathematics & CIS             69
Social Sciences and Law       69
Name: count, dtype: int64

In [27]:
tuition_data["Field of study"].value_counts().sort_index()

Field of study
Architecture & Engineering    172
Arts and Humanities            76
Business & Public Admin        74
Education                      38
Health                        234
Mathematics & CIS              38
Social Sciences and Law        57
Name: count, dtype: int64

In [28]:
income_data["Field of study"].value_counts().sort_index()

Field of study
Architecture & Engineering    128
Arts and Humanities            66
Business & Public Admin        33
Education                      33
Health                         33
Mathematics & CIS              33
Social Sciences and Law        33
Name: count, dtype: int64

In [29]:
labour_data["Field of study"].value_counts().sort_index()

Field of study
Architecture & Engineering    12
Arts and Humanities            6
Business & Public Admin        3
Education                      3
Health                         3
Mathematics & CIS              3
Social Sciences and Law        3
Name: count, dtype: int64

### Normalize and Aggregate Values by `Field of study`
We aggregate data by standardized Field of study to enable consistent comparisons across multiple datasets—labour market outcomes, tuition costs, graduation numbers, and median employment income. Each dataset originally used slightly different field labels, so harmonizing and aggregating them ensures that insights are drawn from aligned categories. This approach allows us to evaluate trends and relationships—such as how employment outcomes or tuition costs vary by field—using a unified framework that avoids mismatches and duplication.

#### Aggregate tution cost per Field of study.

In [30]:
tuition_data["Education"].value_counts()

Education
Graduate         352
Undergraduate    337
Name: count, dtype: int64

In [31]:
tuition_data = tuition_data.groupby(["Year", "Education", "Field of study"], as_index=False).mean(["Tuition"])

In [32]:
tuition_data.head()

,Year,Education,Field of study,Tuition
0,2006,Graduate,Architecture & Engineering,3826.75
1,2006,Graduate,Arts and Humanities,3605.00
2,2006,Graduate,Business & Public Admin,14879.00
3,2006,Graduate,Education,4627.00
4,2006,Graduate,Health,5812.50


#### Aggregate median income per Field of study.

In [33]:
income_data["Education"].value_counts()

Education
Bachelor’s or equivalent    121
Master's or equivalent      121
Doctoral or equivalent      117
Name: count, dtype: int64

In [34]:
income_data = income_data.groupby(
    ["Graduation Year", "Education", "Survey Year", "Field of study"], 
    as_index=False).median(["Median income (2Y)"])

In [35]:
income_data.head()

,Graduation Year,Education,Survey Year,Field of study,Median income
0,2010,Bachelor’s or equivalent,2012,Architecture & Engineering,58650.0
1,2010,Bachelor’s or equivalent,2012,Arts and Humanities,35900.0
2,2010,Bachelor’s or equivalent,2012,Business & Public Admin,56700.0
3,2010,Bachelor’s or equivalent,2012,Education,51200.0
4,2010,Bachelor’s or equivalent,2012,Health,65200.0


#### Aggregate median income per Field of study.

To accurately assess labor market outcomes by field of study, raw counts must be aggregated to calculate population-weighted rates. This function first computes participation, employment, and unemployment rates at the individual record level, then aggregates the data by year and field of study. It recalculates the rates using summed counts to avoid distortions from averaging percentages, ensuring accurate comparisons across fields.

In [36]:
cols_to_convert = [
    'In the labour force',
    'Employed',
    'Unemployed',
    'Not in the labour force'
]

# Clean and convert
for col in cols_to_convert:
    labour_data[col] = labour_data[col].astype(str).str.replace(',', '').str.strip()
    labour_data[col] = pd.to_numeric(labour_data[col], errors='coerce')

In [37]:
def aggregate_labour_rates(df):
    # Step 1: Copy and compute working-age population
    df_ = df.copy()
    df_['Population (25%)'] = df_['In the labour force'] + df_['Not in the labour force']
    
    # Step 2: Recalculate accurate rates for each row (optional)
    df_['Participation rate'] = df_['In the labour force'] / df_['Population (25%)'] * 100
    df_['Employment rate'] = df_['Employed'] / df_['Population (25%)'] * 100
    df_['Unemployment rate'] = df_['Unemployed'] / df_['In the labour force'] * 100
    
    # Step 3: Aggregate raw counts by Year and Field of study
    agg = df_.groupby(['Year', 'Field of study']).agg({
        'In the labour force': 'sum',
        'Employed': 'sum',
        'Unemployed': 'sum',
        'Not in the labour force': 'sum'
    }).reset_index()
    
    # Step 4: Recalculate accurate weighted rates after aggregation
    agg['Population (25%)'] = agg['In the labour force'] + agg['Not in the labour force']
    agg['Participation rate'] = round(agg['In the labour force'] / agg['Population (25%)'] * 100, 2)
    agg['Employment rate'] = round(agg['Employed'] / agg['Population (25%)'] * 100, 2)
    agg['Unemployment rate'] = round(agg['Unemployed'] / agg['In the labour force'] * 100, 2)

    agg = agg.rename(columns={"Employed": "Employed (25%)"})
    
    # Optional: reorder columns for clarity
    return agg[[
        'Year', 'Field of study', 'Population (25%)', 'Employed (25%)',
        'Participation rate', 'Employment rate', 'Unemployment rate'
    ]]

In [38]:
labour_data = aggregate_labour_rates(labour_data)

In [39]:
labour_data.head()

,Year,Field of study,Population (25%),Employed (25%),Participation rate,Employment rate,Unemployment rate
0,2011,Architecture & Engineering,1193185,898205,79.82,75.28,5.69
1,2011,Arts and Humanities,732365,512570,74.95,69.99,6.62
2,2011,Business & Public Admin,1084560,869455,84.43,80.17,5.05
3,2011,Education,814420,561970,71.41,69.00,3.37
4,2011,Health,671290,516825,79.93,76.99,3.67


### Convert Dollar Amounts
According to Statistique Canada, the [2024 CPI](https://www.statcan.gc.ca/en/subjects-start/prices_and_price_indexes/consumer_price_indexes) was 160.9 while it was 151.2 in [2022](https://www150.statcan.gc.ca/n1/daily-quotidien/230117/dq230117b-eng.htm)

In [40]:
def to_current_dollar(amount, cpi, curr_cpi):
    return amount * (curr_cpi / cpi)

In [41]:
tuition_data["Tuition (2024 dollar)"] = to_current_dollar(tuition_data["Tuition"], cpi=151.2, curr_cpi=160.9)

### 2.4. Assign `Education Level Group`
Since the tuition data is less granular and combines all graduate-level education into a single category, we introduce a new variable, `Education Level Group`, to align education levels across datasets and enable consistent mapping.

In [42]:
tuition_data = tuition_data.rename(columns={"Education": "Education Level Group"})

In [43]:
def assign_education_level_group(education):
    if education == "Bachelor's or equivalent":
        return "Undergraduate"
    elif education == "Master's or equivalent" or education == "Doctoral or equivalent":
        return "Graduate"
    raise Exception(f"'{education}' is not recognozed")

### Accumulate number of graduates
To better approximate the number of graduates actively participating in the workforce, we computed a 5-year rolling sum of graduates (`Graduates (5Y)`) for each year. This method assumes that most graduates remain within their field for at least five years before potentially exiting, switching careers, or pursuing further education.

In [44]:
# Ensure the data is sorted by 'Field of study' and 'Year'
graduation_data = graduation_data.sort_values(by=["Field of study", "Year"])

# Compute 5-year cumulative number of grads
graduation_data["Graduates (5Y)"] = (
    graduation_data.groupby("Field of study")["Graduates"]
    .rolling(window=5, min_periods=5)
    .sum()
    .reset_index(level=0, drop=True)
)

## Save data

### Creating Final Datasets
To effectively evaluate the return on investment and real-world value of higher education by field of study, we constructed two aligned datasets. The first dataset combines tuition costs, graduation counts, and median income (five years post-graduation) across all postsecondary education levels, aggregated by Year and Field of study. This enables us to analyze the economic outcomes of different disciplines based on what graduates typically pay, how many enter the workforce, and what they earn after gaining moderate experience. We also include a Census Year column, which links each observation year to the closest available labour market snapshot from the Canadian census (2011, 2016, 2021). The second dataset contains these census-based labour outcomes, including participation, employment, and unemployment rates by Field of study. By aligning both datasets on Census Year and Field of study, we enable cross-sectional analysis of how education supply, cost, and income align with real-world employment conditions over time.

In [45]:
income_data.rename(columns={"Survey Year": "Year"}, inplace=True)

In [46]:
graduation_data["Education"] = graduation_data["Education"].str.replace("’", "'", regex=False)
income_data["Education"] = income_data["Education"].str.replace("’", "'", regex=False)

In [47]:
# Merge graduation data and income data
combined = graduation_data.merge(
    income_data,
    left_on=["Year", "Field of study", "Education"],
    right_on=["Year", "Field of study", "Education"],
    how="left"
)

In [48]:
combined["Education Level Group"] = combined["Education"].apply(assign_education_level_group)

In [49]:
# Merge tuition
combined = combined.merge(
    tuition_data,
    left_on=["Year", "Field of study", "Education Level Group"],
    right_on=["Year", "Field of study", "Education Level Group"],
    how="left"
)

We use a function to assign each observation year to its nearest Canadian census year (2001, 2006, 2011, 2016, 2021, or 2026). Since detailed labour market data is only available for these census years, this mapping ensures that each year in our dataset is aligned with the most representative labour conditions. By grouping years around their closest census year, we maintain temporal coherence and enable meaningful comparisons between graduate outcomes and labour market indicators, even when data sources differ in granularity or update frequency.

In [50]:
def map_to_census_year(year):
    if 2000 <= year <= 2003:
        return 2001
    elif 2004 <= year <= 2008:
        return 2006
    elif 2009 <= year <= 2013:
        return 2011
    elif 2014 <= year <= 2017:
        return 2016
    elif 2018 <= year <= 2022:
        return 2021
    elif 2023 <= year <= 2027:
        return 2026
    else:
        return None

In [51]:
# Add Census Year
combined["Census Year"] = combined["Year"].apply(map_to_census_year)

### Save to CSV File

Save combined education and income data

In [52]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 483 entries, 0 to 482
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   483 non-null    int64  
 1   Field of study         483 non-null    object 
 2   Graduates              483 non-null    float64
 3   Education              483 non-null    object 
 4   Graduates (5Y)         455 non-null    float64
 5   Graduation Year        231 non-null    float64
 6   Median income          231 non-null    float64
 7   Education Level Group  483 non-null    object 
 8   Tuition                357 non-null    float64
 9   Tuition (2024 dollar)  357 non-null    float64
 10  Census Year            483 non-null    int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 41.6+ KB


In [54]:
combined = combined[[
    "Year", "Census Year", "Field of study", "Education", "Education Level Group", 
    "Graduates", "Graduates (5Y)", "Tuition (2024 dollar)", "Median income"
]].rename(columns={
    "Census Year": "Nearest Census Year",
    "Tuition (2024 dollar)": "Tuition"
})

In [62]:
combined.head(3)

,Year,Nearest Census Year,Field of study,Education,Education Level Group,Graduates,Graduates (5Y),Tuition,Median income
0,2000,2001,Architecture & Engineering,Bachelor's or equivalent,Undergraduate,9843.0,NaN,NaN,NaN
1,2000,2001,Architecture & Engineering,Master's or equivalent,Graduate,2415.0,NaN,NaN,NaN
2,2000,2001,Architecture & Engineering,Doctoral or equivalent,Graduate,564.0,NaN,NaN,NaN


In [60]:
combined.to_csv("data/postsecondary_field_outcomes.csv", index=False)

Save labour data

In [55]:
labour_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Year                21 non-null     int64  
 1   Field of study      21 non-null     object 
 2   Population (25%)    21 non-null     int64  
 3   Employed (25%)      21 non-null     int64  
 4   Participation rate  21 non-null     float64
 5   Employment rate     21 non-null     float64
 6   Unemployment rate   21 non-null     float64
dtypes: float64(3), int64(3), object(1)
memory usage: 1.3+ KB


In [56]:
labour_data = labour_data[[
    "Year", "Field of study", "Participation rate",
    "Employment rate", "Unemployment rate", "Employed (25%)"
]].rename(columns={"Year": "Census Year"})

In [63]:
labour_data.head(3)

,Census Year,Field of study,Participation rate,Employment rate,Unemployment rate,Employed (25%)
0,2011,Architecture & Engineering,79.82,75.28,5.69,898205
1,2011,Arts and Humanities,74.95,69.99,6.62,512570
2,2011,Business & Public Admin,84.43,80.17,5.05,869455


In [58]:
labour_data.to_csv("data/labour_market_snapshot.csv", index=False)